In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from urllib.parse import urljoin


In [3]:
url = "https://www.aqaribank.jo/"

bank_name = "Aqari Bank"

folder_name = "AqariBank_Images"
os.makedirs(folder_name, exist_ok=True)

headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, "html.parser")


In [5]:
slides = soup.find_all("img", class_="slide-image")


In [7]:
data = []


In [9]:
for img_tag in slides:
    img_src = img_tag.get("src", "").strip()
    img_alt = img_tag.get("alt", "").strip()

    # محاولة إيجاد رابط المقال إن وجد
    parent_a = img_tag.find_parent("a")
    article_link = ""
    if parent_a and parent_a.has_attr("href"):
        article_link = parent_a["href"].strip()
        if not article_link.startswith("http"):
            article_link = urljoin(url, article_link)

    if img_src:
        # معالجة رابط الصورة لو مش كامل
        full_img_url = img_src
        if not full_img_url.startswith("http"):
            full_img_url = urljoin(url, img_src)

        # تجهيز اسم الصورة
        img_name = img_alt.replace(" ", "_").replace("/", "_") or "no_name"
        img_extension = full_img_url.split(".")[-1].split("?")[0]
        img_filename = f"{img_name}.{img_extension}"
        img_path = os.path.join(folder_name, img_filename)

        # تحميل الصورة
        try:
            img_response = requests.get(full_img_url)
            img_response.raise_for_status()
            with open(img_path, "wb") as f:
                f.write(img_response.content)
            print(f"✔️ تم تحميل الصورة: {img_filename}")
        except Exception as e:
            print(f"❌ فشل تحميل الصورة: {full_img_url} - السبب: {e}")

        # إضافة البيانات للـ DataFrame
        data.append({
            "Bank_Name": bank_name,
            "Image_Name": img_filename,
            "Image_Link": full_img_url,
            "Article_Link": article_link
        })

In [11]:
df = pd.DataFrame(data)
df

""
